# GTEx v8 RNA-seq data processing for eQTLs : Exploratory analysis

This notebook outlines how the expression matrices are handled prior to the eQTL mapping, and also include some exploratory analysis on the data. In particular, we focus on hidden structures within the expression data, and how we correct for the relevant covariates, both known and inferred.

On Della, the RNA-seq project directory is located at:

<code>/tigress/BEE/RNAseq/</code>

In [2]:
%%bash
# export proj='/tigress/BEE/RNAseq/'
# The local directory used for development and Github commits are located at:
export proj='/Users/brian_jo/Desktop/Project/RNAseq_pipeline/'

## Creating reference tables - samples, subjects and tissues

I wrote a script that reads in the GTEx sample and subject-level annotation files, downloaded from dbGaP and located at:

<code>/tigress/BEE/gtex/dbGaP-7716/57610/gtex/exchange/GTEx_phs000424/exchange/analysis_releases/GTEx_Analysis_2017-06-05_v8/sample_annotations/</code>

and writing three tables that are organized at sample, subject and tissue levels:

<code>/tigress/BEE/RNAseq/Data/Resources/gtex/tables/v8/</code>

The code can be found at:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/misc/silver/v8_organize_tissues_reference_tables.R</code>

In [ ]:
%%bash

Rscript $proj/Scripts/misc/silver/v8_organize_tissues_reference_tables.R

In [5]:
%%bash

echo 'Sample table'
head -n 10 $proj/Data/Resources/gtex/tables/v8/sample_table.txt
echo ' '
echo 'Subject table'
head -n 10 $proj/Data/Resources/gtex/tables/v8/subject_table.txt
echo ' '
echo 'Tissue table'
head -n 10 $proj/Data/Resources/gtex/tables/v8/tissue_table.txt
echo ' '
echo 'Number of samples + 1'
wc -l $proj/Data/Resources/gtex/tables/v8/sample_table.txt
echo ' '
echo 'Number of subjects + 1'
wc -l $proj/Data/Resources/gtex/tables/v8/subject_table.txt
echo ' '
echo 'Number of tissues + 1'
wc -l $proj/Data/Resources/gtex/tables/v8/tissue_table.txt

Sample table
SAMPID	SMTSD	SUBJID	SEX	RACE	AGE	BMI	tissue_name
GTEX-1117F-0226-SM-5GZZ7	Adipose - Subcutaneous	GTEX-1117F	2	2	66	32.12	adiposesubcutaneous
GTEX-1117F-0426-SM-5EGHI	Muscle - Skeletal	GTEX-1117F	2	2	66	32.12	muscleskeletal
GTEX-1117F-0526-SM-5EGHJ	Artery - Tibial	GTEX-1117F	2	2	66	32.12	arterytibial
GTEX-1117F-0626-SM-5N9CS	Artery - Coronary	GTEX-1117F	2	2	66	32.12	arterycoronary
GTEX-1117F-0726-SM-5GIEN	Heart - Atrial Appendage	GTEX-1117F	2	2	66	32.12	heartatrialappendage
GTEX-1117F-1326-SM-5EGHH	Adipose - Visceral (Omentum)	GTEX-1117F	2	2	66	32.12	adiposevisceralomentum
GTEX-1117F-2226-SM-5N9CH	Ovary	GTEX-1117F	2	2	66	32.12	ovary
GTEX-1117F-2426-SM-5EGGH	Uterus	GTEX-1117F	2	2	66	32.12	uterus
GTEX-1117F-2526-SM-5GZY6	Vagina	GTEX-1117F	2	2	66	32.12	vagina
 
Subject table
SUBJID	SEX	AGE	RACE	ETHNCTY	BMI	num_samples
GTEX-1117F	2	66	2	0	32.12	13
GTEX-111CU	1	57	3	0	33.57	19
GTEX-111FC	1	61	3	0	25.06	16
GTEX-111VG	1	63	3	0	29.53	11
GTEX-111YS	1	62	3	0	30.78	25
GTEX-1122O	2	64	

There are <b>17395</b> samples coming from <b>948</b> subjects, over <b>54</b> tissues - certainly an extensive RNA-seq dataset by today's standards.

## Organizing the GTEx gct expression matrices and normalizing them

<code>Note: This part of the notebook is obsolete - we simply take the normalized version of the expression provided by the GTEx consortium.</code>

The GTEx v8 expression gct files are also downloaded from dbGaP and are located at:

<code>tigress/BEE/gtex/dbGaP-7716/56946/gtex/exchange/GTEx_phs000424/exchange/analysis_releases/GTEx_Analysis_2017-06-05_v8/rna_seq</code>

We obtain the main gct files, and separate out by tissues first, for four sets of expression matrices that have been obtained using [RSEMv1.3.0](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-323):

- Gene TPM
- Gene Counts
- Transcript TPM
- Transcript Counts

The script for doing this is located at:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/misc/silver/v8_separate_expression_matrices.py</code>

The wrapper script for the cluster:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/misc/silver/v8_separate_expression_matrices_wrapper.py</code>

In [ ]:
%%bash

python $proj/Scripts/misc/silver/v8_separate_expression_matrices_wrapper.py

As a result of running this, the separated expression matrices are deposited at:

<code>/tigress/BEE/gtex/data/phenotype/expression/expression_matrices/v8/</code>

I've organized the directory such that there are 5 subdirectories:

- dbGaP - Copy of the original files downloaded from dbGaP
- gene_list - List of Genes that are used for downstream analysis, after filtering
- log_transform - Log-transformed version of the count matrices
- quantile_norm - Quantile-normalized version of the TPM matrices
- raw - Tissue-separated version of the dbGaP gct files

The dbGaP gene matrices have <b>58219</b> genes, and the transcript matrices have <b>199324</b> transcripts.

The next step is normalization. The normalization script does the following:

- Filter gene and transcript tables based on minimal gene expression - our threshold is at least 0.1 TPM for at least 10 individuals. This criteria only applies to genes, and in the transcript table, all transcripts that belong to the filtered gene list are included.
- Quantile-normalize the TPM matrices. There are two types of QN procedure:

<code>QN the entire expression profile (all samples, all genes, ties broken by averaging), and then QN for each gene (ties broken randomly)</code>

<code>QN for each sample (ties broken by averaging), and then QN for each gene (ties broken randomly)</code>

- Log-transform the Count matrices (Log2 transform the TPM values + 1, which maps 0 to 0 and 1 to 1)

The script for doing this is located at:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/processing/silver/v8_normalize_matrices.py</code>

The wrapper script for the cluster:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/processing/silver/v8_normalize_matrices_wrapper.py</code>

In [ ]:
%%bash

python $proj/Scripts/processing/silver/v8_normalize_matrices_wrapper.py

It is worth noting that in order to bring up the list of transcripts that belong to a gene, I used [GENCODE v.26](https://www.gencodegenes.org/releases/26lift37.html). The converted Python OrderedDict is saved at:

<code>/tigress/BEE/RNAseq/Data/Resources/annotations/silver/gencode.v26.transcripts</code>

- The resulting quantile-normalized TPM matrices are saved under the <code>quantile_norm</code> subdirectory, with the postfix <code>profile_norm.txt</code> meaning the first QN method, and <code>sample_norm.txt</code> meaning the second.
- The resulting log-transformed Count matrices are saved under the <code>log_transform</code> subdirectory.
- The filtered gene list and transcript list are saved under the <code>gene_list</code> subdirectory.


## Exploring various normalization techniques for expression matrices

Now that we have the expression values in the quantile-normalized form, we can try out various factor analysis techniques that seek to find and correct for unknown factors in the expression profile, which will (hopefully) also include contributions from genotype PCs (roughly accounting for population structure) and other known covariates (other potential confounders).

I will try out the following three methods:

- [SFAMix - Sparse and Dense Factor Analysis](https://arxiv.org/abs/1310.4792) - the code can be downloaded at https://github.com/chuangao/SFAmix
- [PEER - Probabilistic Estimation of Expression Residuals](https://www.ncbi.nlm.nih.gov/pubmed/22343431) - the code can be downloaded at https://github.com/PMBio/peer
- [SVA - Surrogate Variable Analysis](http://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.0030161) - The code is in the form of an [R package](https://www.bioconductor.org/packages/devel/bioc/vignettes/sva/inst/doc/sva.pdf)


### SFAmix - output steps in increments of 50 until convergence (or 5000 steps, whichever comes first)

In [ ]:
%%bash

python $proj/Scripts/processing/covariates_exploratory/v8_sfamix_incremental_wrapper.py

The script for doing this is located at:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/processing/covariates_exploratory/v8_sfamix_incremental.R</code>

The wrapper script for the cluster:

<code>https://github.com/bee-hive/RNAseq_pipeline/blob/master/Scripts/processing/covariates_exploratory/v8_sfamix_incremental_wrapper.py</code>

The outputs of this step can be found at:
<code>
/tigress/BEE/RNAseq/Output/processing/exploratory/v8/sfamix/GTEx_Analysis_v8/
</code>